<a href="https://colab.research.google.com/github/YangYangJiJi/2025_Artificial_Intelligent/blob/main/puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpleai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=101052 sha256=9724efed242c65e01ffac185ac2f2fbc6e6a447447740ea3f1c04ed5e8a6d677
  Stored in directory: /root/.cache/pip/wheels/ec/02/a7/f0077617a5f73eb1c52e45f12a9da3f0bafff3902bcd91766f
Successfully built simpleai


In [2]:
from simpleai.search.utils import FifoList, BoundedPriorityQueue, LifoList
from simpleai.search.models import (SearchNode, SearchNodeHeuristicOrdered,
                                    SearchNodeStarOrdered,
                                    SearchNodeCostOrdered)
from simpleai.search import SearchProblem

def greedy(problem, graph_search=False, viewer=None):
    '''
    Greedy search.

    If graph_search=True, will avoid exploring repeated states.
    Requires: SearchProblem.actions, SearchProblem.result,
    SearchProblem.is_goal, SearchProblem.cost, and SearchProblem.heuristic.
    '''
    return _search(problem,
                   BoundedPriorityQueue(),
                   graph_search=graph_search,
                   node_factory=SearchNodeHeuristicOrdered,
                   graph_replace_when_better=True,
                   viewer=viewer)

def astar(problem, graph_search=False, viewer=None):
    '''
    A* search.

    If graph_search=True, will avoid exploring repeated states.
    Requires: SearchProblem.actions, SearchProblem.result,
    SearchProblem.is_goal, SearchProblem.cost, and SearchProblem.heuristic.
    '''
    return _search(problem,
                   BoundedPriorityQueue(),
                   graph_search=graph_search,
                   node_factory=SearchNodeStarOrdered,
                   graph_replace_when_better=True,
                   viewer=viewer)

def _search(problem, fringe, graph_search=False, depth_limit=None,
            node_factory=SearchNode, graph_replace_when_better=False,
            viewer=None):
    '''
    Basic search algorithm, base of all the other search algorithms.
    '''
    if viewer:
        viewer.event('started')

    memory = set()
    initial_node = node_factory(state=problem.initial_state,
                                problem=problem)
    fringe.append(initial_node)

    while fringe:
        if viewer:
            viewer.event('new_iteration', fringe.sorted())

        node = fringe.pop()
        memory.add(node.state)

        if problem.is_goal(node.state):
            if viewer:
                viewer.event('chosen_node', node, True)
                viewer.event('finished', fringe.sorted(), node, 'goal found')
            print('확장 노드 수 : '+ str(len(memory)))
            print('생성 노드 수 : '+ str(len(memory) + len(fringe)))
            print('해 길이 : '+ str(len(node.path())))
            return node
        else:
            if viewer:
                viewer.event('chosen_node', node, False)


        if depth_limit is None or node.depth < depth_limit:
            expanded = node.expand()
            if viewer:
                viewer.event('expanded', [node], [expanded])

            for n in expanded:
                if graph_search:
                    others = [x for x in fringe if x.state == n.state]
                    assert len(others) in (0, 1)
                    if n.state not in memory and len(others) == 0:
                        fringe.append(n)
                    elif graph_replace_when_better and len(others) > 0 and n < others[0]:
                        fringe.remove(others[0])
                        fringe.append(n)
                else:
                    fringe.append(n)

    if viewer:
        viewer.event('finished', fringe.sorted(), None, 'goal not found')

In [33]:
# Class containing methods to solve the puzzle
class PuzzleSolver(SearchProblem):
    # Action method to get the list of the possible
    # numbers that can be moved in to the empty space
    def actions(self, cur_state):
        rows = string_to_list(cur_state)
        row_empty, col_empty = get_location(rows, 'e')

        actions = []
        if row_empty > 0:
            actions.append(rows[row_empty - 1][col_empty])
        if row_empty < 2:
            actions.append(rows[row_empty + 1][col_empty])
        if col_empty > 0:
            actions.append(rows[row_empty][col_empty - 1])
        if col_empty < 2:
            actions.append(rows[row_empty][col_empty + 1])

        return actions

    # Return the resulting state after moving a piece to the empty space
    def result(self, state, action):
        rows = string_to_list(state)
        row_empty, col_empty = get_location(rows, 'e')
        row_new, col_new = get_location(rows, action)

        rows[row_empty][col_empty], rows[row_new][col_new] = \
                rows[row_new][col_new], rows[row_empty][col_empty]

        return list_to_string(rows)

    # Returns true if a state is the goal state
    def is_goal(self, state):
        return state == GOAL

    # number of misplaced tiles
    '''def heuristic(self, state):
        rows = string_to_list(state)

        wrong = 0

        for number in '12345678':
            row_new, col_new = get_location(rows, number)
            row_new_goal, col_new_goal = goal_positions[number]

            wrong += 0 if (row_new == row_new_goal) and (col_new == col_new_goal) else 1

        return wrong'''

    # Returns an estimate of the distance from a state to
    # the goal using the manhattan distance
    def heuristic(self, state):
        rows = string_to_list(state)

        distance = 0

        for number in '12345678':
            row_new, col_new = get_location(rows, number)
            row_new_goal, col_new_goal = goal_positions[number]

            distance += abs(row_new - row_new_goal) + abs(col_new - col_new_goal)

        return distance


# Convert list to string
def list_to_string(input_list):
    return '\n'.join(['-'.join(x) for x in input_list])

# Convert string to list
def string_to_list(input_string):
    return [x.split('-') for x in input_string.split('\n')]

# Find the 2D location of the input element
def get_location(rows, input_element):
    for i, row in enumerate(rows):
        for j, item in enumerate(row):
            if item == input_element:
                return i, j

# Final result that we want to achieve
GOALS = [
'1-2-3\n8-e-4\n7-6-5',
'1-2-3\n4-5-6\n7-8-e',
'6-e-1\n8-5-3\n4-7-2',
'5-6-7\n2-e-3\n8-1-4',
'7-6-3\n5-8-2\n1-4-e',
'5-8-e\n7-3-2\n1-6-4',
'2-3-6\n5-7-4\n8-1-e',
'1-3-e\n8-2-4\n5-6-7',
'3-1-8\n6-7-4\ne-5-2',
'4-2-5\ne-1-3\n6-8-7'
]

# Starting point
INITIALS = [
'2-e-3\n1-8-4\n7-6-5',
'1-e-2\n6-3-4\n7-5-8',
'8-3-5\n4-6-e\n2-7-1',
'e-8-6\n4-3-1\n5-2-7',
'7-1-e\n2-8-6\n5-4-3',
'5-2-1\n6-4-7\n3-e-8',
'e-7-1\n5-6-2\n8-4-3',
'3-e-8\n2-6-4\n1-7-5',
'6-e-2\n7-8-1\n4-3-5',
'2-7-e\n8-6-1\n4-5-3'
]

k = 9
GOAL, INITIAL = GOALS[k], INITIALS[k]


# Create a cache for the goal position of each piece
goal_positions = {}
rows_goal = string_to_list(GOAL)
for number in '12345678':
    goal_positions[number] = get_location(rows_goal, number)


# Create the solver object
result = greedy(PuzzleSolver(INITIAL), graph_search=True)


# Print the results
for i, (action, state) in enumerate(result.path()):
    print()
    if action == None:
        print('Initial configuration')
    elif i == len(result.path()) - 1:
        print('After moving', action, 'into the empty space. Goal achieved!')
    else:
        print('After moving', action, 'into the empty space')

    print(state)



확장 노드 수 : 262
생성 노드 수 : 449
해 길이 : 98

Initial configuration
2-7-e
8-6-1
4-5-3

After moving 7 into the empty space
2-e-7
8-6-1
4-5-3

After moving 2 into the empty space
e-2-7
8-6-1
4-5-3

After moving 8 into the empty space
8-2-7
e-6-1
4-5-3

After moving 6 into the empty space
8-2-7
6-e-1
4-5-3

After moving 1 into the empty space
8-2-7
6-1-e
4-5-3

After moving 3 into the empty space
8-2-7
6-1-3
4-5-e

After moving 5 into the empty space
8-2-7
6-1-3
4-e-5

After moving 4 into the empty space
8-2-7
6-1-3
e-4-5

After moving 6 into the empty space
8-2-7
e-1-3
6-4-5

After moving 8 into the empty space
e-2-7
8-1-3
6-4-5

After moving 2 into the empty space
2-e-7
8-1-3
6-4-5

After moving 1 into the empty space
2-1-7
8-e-3
6-4-5

After moving 8 into the empty space
2-1-7
e-8-3
6-4-5

After moving 6 into the empty space
2-1-7
6-8-3
e-4-5

After moving 4 into the empty space
2-1-7
6-8-3
4-e-5

After moving 8 into the empty space
2-1-7
6-e-3
4-8-5

After moving 1 into the empty space
2-e-